<a href="https://colab.research.google.com/github/jyothish-mohan/ResNet18_testing-_with_Collab_GPU/blob/main/Collab_GPU_Testing_with_ResNet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist.load_data()
(X_train, y_train), (X_test, y_test) = fashion_mnist

X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

X_train = X_train.reshape(-1,28,28,1)

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
from functools import partial

In [ ]:
DefaultConv2D = partial(keras.layers.Conv2D,strides=1,padding="SAME",kernel_size=3,use_bias=False)

In [ ]:
# Creating Residual Block
class ResidualUnit(keras.layers.Layer):
    def __init__(self,filters,strides=1,activation='relu',**kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [DefaultConv2D(filters,strides=strides),keras.layers.BatchNormalization(),self.activation,
                            DefaultConv2D(filters),keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [DefaultConv2D(filters,kernel_size=1,strides=strides),keras.layers.BatchNormalization()]
        
    def call(self,inputs):
        Z = inputs
        for layers in self.main_layers:
            Z = layers(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
            
        return self.activation(Z+skip_Z)

In [ ]:
model = keras.models.Sequential()
#Conv layer
model.add(DefaultConv2D(64,kernel_size=7,strides=2,input_shape=[28,28,1]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=3,strides=2,padding="SAME"))
#Deep Residual Network
prev_filters = 64
for filters in [64]*2 + [128]*2 + [256]*2 + [512]*2 :
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters,strides=strides))
    prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10,activation="softmax"))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,epochs=15)

Epoch 1/15
1875/1875 [==============================] - 74s 39ms/step - loss: 0.3023 - accuracy: 0.8893
Epoch 2/15
1875/1875 [==============================] - 73s 39ms/step - loss: 0.2593 - accuracy: 0.9025
Epoch 3/15
1875/1875 [==============================] - 73s 39ms/step - loss: 0.2271 - accuracy: 0.9144
Epoch 4/15
1875/1875 [==============================] - 73s 39ms/step - loss: 0.2034 - accuracy: 0.9238
Epoch 5/15
1875/1875 [==============================] - 74s 39ms/step - loss: 0.1799 - accuracy: 0.9314
Epoch 6/15
1875/1875 [==============================] - 74s 39ms/step - loss: 0.1642 - accuracy: 0.9380
Epoch 7/15
1875/1875 [==============================] - 74s 39ms/step - loss: 0.1446 - accuracy: 0.9447
Epoch 8/15
1875/1875 [==============================] - 73s 39ms/step - loss: 0.1311 - accuracy: 0.9498
Epoch 9/15
1875/1875 [==============================] - 74s 39ms/step - loss: 0.1184 - accuracy: 0.9548
Epoch 10/15
1875/1875 [==============================] - 74s 39m

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat","Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
pred_data = X_test[:4]
pred_data = pred_data.reshape(-1,28,28,1)
y_pred = model.predict_classes(pred_data)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
np.array(class_names)[y_pred]

array(['Ankle boot', 'Pullover', 'Trouser', 'Trouser'], dtype='<U11')

In [ ]:
np.array(class_names)[y_test[:4]]

array(['Ankle boot', 'Pullover', 'Trouser', 'Trouser'], dtype='<U11')